In [1]:
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from openai import OpenAI
import project_fuctions as pf

1. Set parameters

In [3]:
# API key and url for Google map POI scraping
API_KEY = Google_place_key
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

# The country to process and file directions
country = 'Korea' # If Abbre. of country is commonly used, joined names with '|'. For exampe: United Kingdom|UK
route_input = route_input # The attraction list for POI retrieving
route_export = route_output
route_POI_raw_data = route_raw_data

# Threshold to filter POIs
review_threshold = 0

OpenAI_key = OpenAI_key

2. Get POI list througn Google API

In [ ]:
POI_to_get = pd.read_csv(route_input)
POI_places = POI_to_get.values.tolist()
POI_result = pd.DataFrame()
failed = []
n = 0
page_token = None
for place in POI_places:
    print(f'request No.{n} {place[0]}, {place[1]}')
    params = {
    'query': f'{place[0]}, {place[1]}',
    'key': API_KEY,
        #'pagetoken' : page_token,
        #'type' : "amusement_park",
        }
    result = requests.get(url, params=params)
    #Convert data into json type first and dataframe then
    result_js = result.json()
    result_df = pd.json_normalize(result_js['results'])
    if len(result_df) > 0: 
        print(f'{result_df.loc[0]["name"]}')
        result_df['Origin'] = pd.Series([place for x in range(len(result_df.index))])
    #if len(result_df) == 0
    #Check if there is next page and send the token to params
    #if 'next_page_token' in result_js:
    #   page_token = result_js['next_page_token']
    #else:
    #    page_token = None
        POI_result = pd.concat([POI_result,result_df])
        print(f'No.{n} successed')
    else: 
        failed.append(n)
        print(f'No.{n} failed')
    n = n+1
    time.sleep(11)

3. Update the admission and category by dynamically scraping Google Map

In [ ]:
POI_list = pf.cleanup(POI_result,review_threshold,country)

driver = webdriver.Chrome()
url_part = 'https://www.google.com/maps/place/?q=place_id:'
driver.execute_script("document.body.style.zoom='50%'")

hotel_path = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/div[1]/div[2]/div/div/span/span/span/span[2]/span/span'

for index, item in POI_list.iterrows():
    url = url_part + item['place_id']
    driver.get(url)
 
    try:
        WebDriverWait(driver, 2, 0.2).until(EC.visibility_of_element_located((By.CLASS_NAME,"drwWxc")),"No value")       
        admission = driver.find_element(By.CLASS_NAME,"drwWxc").text
    except:
        admission = "Unknown"
    
    try:
        place_type = driver.find_element(By.CLASS_NAME,'DkEaL').text
    except:
        try:
            WebDriverWait(driver, 1, 0.2).until(EC.visibility_of_element_located((By.XPATH,hotel_path)),"No value")       
            place_type = driver.find_element(By.XPATH,hotel_path).text
        except:
            place_type = "Unknown"

    try:
        description = driver.find_element(By.CLASS_NAME,'PYvSYb').text
    except:
        description = "Unknown"
    
    POI_list.loc[index, 'Place Type'] = place_type
    POI_list.loc[index, 'Free or Paid'] = admission
    POI_list.loc[index, 'Description'] = description
    print(f"{index} - {item['name']} is {place_type}, its admission is {admission}")
    if (index+1)%100 == 0: POI_list.to_csv(route_export, index=False)

4. Categorize BU Level 3

In [ ]:
label_list = '["Ancient Ruins","Aquariums","Cable Car & Gondola & Skywheels", "Castles & Palaces", "Cathedral & Churches", "Farms", "Gardens & Parks", "Museums & Galleries", "Natural Landscape", "Observation Decks & Towers", "Playgrounds", "Temples & Shrines", "Theme Park", "Villages", "Shows & Musicals", "Water Parks", "Zoos & Animal Parks", "Sports", "Not Attraction - Lodging", "Not Attraction - F&B", "Not Attraction - Shopping", "Not Attraction - Transportation","not sure"]'
for index, row in POI_list.iterrows():
    attraction = row.loc['name']
    google_tag = row.loc['Place Type']
    description = row.loc['Description']
    prompt = f"""
    Your task is to categorize the attaction delimited by <> with only one label from the label list delimited by <tag></tag>.
    
    Followings are instructions:
    1. The description of the attraction is delimited by %%, which you can take for reference, but do not return content in it!
    2. You can refer to the tags given by google map delimited by ```, but do not use any of those tags!
    3. The response should be exactly from label list delimited by <tag></tag>, no punctuation included.

    attraction: <{attraction}>
    label_list: <tag>{label_list}</tag>
    attraction description: %%{description}%%
    Tag List: ```{google_tag}```
    """
    response = pf.get_completion(prompt)
    POI_list.loc[index, 'BU Level 3'] = response
    print(f'{index}: {attraction} labeled: {response}')
    time.sleep(0.2)

In [ ]:
POI_list['Gmap URL'] = url_part + POI_list['place_id']
column_order = ['state','city','name','rating','user_ratings_total','Free or Paid','Place Type','Gmap URL','BU Level 3','formatted_address','place_id','types','Description']
POI_list = POI_list[column_order]
POI_list.to_csv(route_export, index=False)
POI_list['Gmap URL'].to_csv('/Users/glen.gu/Documents/Python Project/Acquisition Plan/Frence/Lang/URL_FR_to_apify_missing_1206.csv', index=False, header=False)